# CSS Lab: Networks

In [ ]:
import json
import math
import networkx as nx
import networkx.algorithms.community as nxcom
import pandas as pd
import visJS2jupyter.visJS_module as vjs

## Build and visualize a network

In [ ]:
def load_lost_circles_json(in_file):
    with open(in_file) as f:
        raw = json.load(f)
    id_to_name = dict((i, datum["name"]) for i, datum in enumerate(raw['nodes']))
    edges = [(datum["source"], datum["target"]) for datum in raw['links']]
    return id_to_name, edges

In [ ]:
in_file = "external/LostCircles/sample.json"
id_to_name, edges = load_lost_circles_json(in_file)

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)

In [ ]:
def visualize(G):
    nodes = list(G.nodes())
    edges = list(G.edges())
    pos = nx.spring_layout(G, k=3/math.sqrt(len(nodes)), scale=7000)
    nodes_dict = [{
        "id": id_to_name[n],
        "x": pos[n][0],
        "y": pos[n][1],
        "node_size_field": "node_size",
        "node_size": 5}
        for n in nodes]
    node_map = dict(zip(nodes,range(len(nodes)))) # map to indices for source/target in edges
    edges_dict = [{
        "source": node_map[edges[i][0]],
        "target": node_map[edges[i][1]],
        "title":'test',
        "color": '#d0d0d0'}
        for i in range(len(edges))]
    return vjs.visjs_network(
    nodes_dict, edges_dict,
    node_size_multiplier=10.0)

In [ ]:
visualize(G)

## Calculate centralities

In [ ]:
degree = nx.degree_centrality(G)
betweenness = nx.betweenness_centrality(G)
eigenvector = nx.eigenvector_centrality(G)
closeness = nx.closeness_centrality(G)

In [ ]:
df = pd.DataFrame({
    "id": nodes,
    "label": [id_to_name[n] for n in nodes],
    "degree": [degree[n] for n in nodes],
    "betweenness": [betweenness[n] for n in nodes],
    "eigenvector": [eigenvector[n] for n in nodes],
    "closeness": [closeness[n] for n in nodes]}).set_index("id")
df.sort_values("eigenvector", ascending=False).head()

## Find communities

In [ ]:
community = nxcom.girvan_newman(G)

## References

[RLWGBF2017] Rosenthal, S. B., Len, J., Webster, M., Gary, A., Birmingham, A., & Fisch, K. M. (2017). Interactive network visualization in Jupyter notebooks: visJS2jupyter. Bioinformatics.